In [20]:
import pandas as pd
import numpy as np
import math

### Carga dos dados

In [115]:
# definir arquivo Pickle de entrada
#arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

df = pd.read_pickle(arquivo)
df.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,16078311,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,8698204,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,4402784,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,1,3956710,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
ANA,0,3643117,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [116]:
df.shape

(99998, 18)

### Expansão dos dados agregados

In [117]:
%%time
df2 = df.head(50000).copy()
#df2 = df.copy()
menor = df2['QTDE'].min()
maior = df2['QTDE'].max()
print(df2.shape[0], menor, maior, math.ceil(np.sqrt(maior / menor)))
#df2['TAM'] = df2['QTDE'].apply(lambda x: math.ceil(np.sqrt(x / menor)))
df2['TEMP'] = df2[['SEXO', 'QTDE']].apply(
    lambda x: [x[0]] * math.ceil(np.sqrt(x[1] / menor)), axis=1)
dfe = df2.explode('TEMP').drop(['TEMP'], axis=1) #.reset_index()
dfe.head(1000)

50000 89 16078311 426
CPU times: user 565 ms, sys: 16 ms, total: 581 ms
Wall time: 579 ms


,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,16078311,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,16078311,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,16078311,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,16078311,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,16078311,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ANTONIO,1,3956710,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
ANTONIO,1,3956710,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
ANTONIO,1,3956710,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0


In [118]:
# usar os dados expandidos invés dos agregados
dfo = df # guardar o dataframe original
df = dfe # apontar para o dataframe expandido

### Decomposição com PCA

In [119]:
#from sklearn.decomposition import PCA
#X_reduced = PCA(n_components=4).fit_transform(df.drop(['SEXO'], axis=1))
#X_reduced[:5]

In [120]:
#X = pd.DataFrame(X_reduced, index=df.index)
#y = df['SEXO'] # apenas a coluna alvo
#print('Forma dos dados:', X.shape, y.shape)

### Uso dos dados originais

In [121]:
X = df.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (187327, 16) (187327,)


In [122]:
X.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0


In [123]:
y.head()

NOME
MARIA    0
MARIA    0
MARIA    0
MARIA    0
MARIA    0
Name: SEXO, dtype: uint8

In [124]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [125]:
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

models = {}

def evaluate_model(name, model, X=X, y=y):

  start = datetime.now()
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  results = cross_val_score(model, X, y, cv=kfold,
                            scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  score = results.mean() * 100
  stddev = results.std() * 100

  models[name] = (model, score, stddev, elapsed)
  print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
        (score, stddev, elapsed))
  return score, stddev, elapsed

In [126]:
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV

# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [127]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    random_state=42, multi_class='auto', 
    max_iter=1000, solver='liblinear', C=0.1) # 93.10
evaluate_model('LR', model)

params = dict(
    solver=['liblinear','lbfgs'], 
    C=np.logspace(-3,3,7)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


LogisticRegression(C=0.1, max_iter=1000, random_state=42, solver='liblinear') 
Cross-Validation Score: 82.89 (+/- 0.24) [ 2271 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.2s finished


In [128]:
# Linear SVM
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=42, max_iter=1000, C=0.1) # 93.07
evaluate_model('LSVM', model)

#from scipy.stats import reciprocal, uniform
#params = dict(C=uniform(1, 10))
params = dict(
    C=np.logspace(-4,3,8)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


LinearSVC(C=0.1, random_state=42) 
Cross-Validation Score: 83.15 (+/- 0.36) [35470 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   35.4s finished


In [129]:
# Linear Discriminant Analysis (LDA)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis(solver='svd') # 93.20
evaluate_model('LDA', model)

params = dict(
    solver=['svd','lsqr','eigen']
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


LinearDiscriminantAnalysis() 
Cross-Validation Score: 83.33 (+/- 0.25) [  951 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished


In [130]:
# Quadratic Discriminant Analysis (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model = QuadraticDiscriminantAnalysis() # 44.99
evaluate_model('QDA', model)

params = dict()
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


QuadraticDiscriminantAnalysis() 
Cross-Validation Score: 53.75 (+/- 0.14) [  454 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.4s finished


In [131]:
# Naïve Bayes - Gaussian
from sklearn.naive_bayes import GaussianNB

model = GaussianNB(priors=None, var_smoothing=1.0) # 92.95
evaluate_model('GNB', model)

params = dict(
    var_smoothing=np.logspace(0,-9,num=10)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


GaussianNB(var_smoothing=1.0) 
Cross-Validation Score: 82.80 (+/- 0.24) [  306 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [132]:
# Naïve Bayes - Bernoulli
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB() # 52.24
evaluate_model('BNB', model)

#params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


BernoulliNB() 
Cross-Validation Score: 53.50 (+/- 0.09) [  351 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


(53.50430073460119, 0.09206369192591034, 351)

In [133]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(
    random_state=42, criterion='gini', max_depth=17) # 97.23
evaluate_model('DT', model)

params = dict(
    criterion=['gini','entropy'],
    max_depth=[3,5,7,11,13,17,19]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


DecisionTreeClassifier(max_depth=17, random_state=42) 
Cross-Validation Score: 93.71 (+/- 0.10) [  619 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


In [134]:
# Ensemble - Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    random_state=42, max_features='auto', n_estimators=100) # 99.80
evaluate_model('RF', model)

params = dict(
    n_estimators=[10,50,100,500], 
    max_features=['auto','sqrt','log2']
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


RandomForestClassifier(random_state=42) 
Cross-Validation Score: 94.15 (+/- 0.09) [ 9996 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    9.9s finished


In [135]:
# Ensemble - Ada Boost
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(
    DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17), 
    n_estimators=3) # 99.76
evaluate_model('ABDT', model)

params = dict(
    n_estimators=[1,3,5,7,9,11,13]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_depth=17,
                                                         random_state=42),
                   n_estimators=3) 
Cross-Validation Score: 93.94 (+/- 0.09) [ 1551 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.4s finished


In [136]:
# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=42, max_iter=100, tol=0.01) # 91.59
evaluate_model('SGD', model)

params = dict(
    max_iter=[100, 200, 350, 500, 1000], 
    tol=[0.01, 0.1, 1.0]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


SGDClassifier(max_iter=100, random_state=42, tol=0.01) 
Cross-Validation Score: 74.66 (+/- 9.17) [ 1202 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.1s finished


In [137]:
# Perceptron
from sklearn.linear_model import Perceptron

model = Perceptron(random_state=42, max_iter=750, tol=0.1) # 90.87
evaluate_model('PCT', model)

params = dict(
    max_iter=[100, 200, 350, 500, 750, 1000], 
    tol=[0.1, 0.01, 0.001]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Perceptron(max_iter=750, random_state=42, tol=0.1) 
Cross-Validation Score: 65.79 (+/- 6.51) [  563 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


In [138]:
# Extreme Gradient Boosting Machine (XGBM)
from xgboost import XGBClassifier
#!pip install xgboost

model = XGBClassifier(
    use_label_encoder=False, eval_metric='logloss',
    max_depth=9, min_child_weight=1, gamma=0.4, 
    subsample=0.9, colsample_bytree=0.6, reg_alpha=1e-05) # 99.20
evaluate_model('XGB', model)

params = dict(max_depth=range(3,10,2), min_child_weight=range(1,6,2)) # 99.85
#params = dict(gamma=[i/10.0 for i in range(0,5)]) # 99.85
#params = dict(subsample=[i/10.0 for i in range(6,10)], colsample_bytree=[i/10.0 for i in range(6,10)]) # 99.85
#params = dict(reg_alpha=[1e-5, 1e-2, 0.1, 1, 100]) # 99.85
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.6,
              eval_metric='logloss', gamma=0.4, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=1e-05, reg_lambda=None,
              scale_pos_weight=None, subsample=0.9, tree_method=None,
              use_label_encoder=False, validate_parameters=None,
              verbosity=None) 
Cross-Validation Score: 94.22 (+/- 0.05) [180275 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.0min finished


In [139]:
# LightGBM
from lightgbm import LGBMClassifier
#!pip install lightgbm

model = LGBMClassifier(
    num_leaves=97, max_depth=11, 
    learning_rate=0.1, feature_fraction=0.5) # 97.09
evaluate_model('LGBM', model)

#params = dict(num_leaves=range(10,100), max_depth=range(1,30)) # 99.85
#params = dict(learning_rate=[0.01,0.05,0.1,0.5]) # 99.85
params = dict(feature_fraction=[0.1,0.25,0.5,0.75,1.0]) # 99.87
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


LGBMClassifier(feature_fraction=0.5, max_depth=11, num_leaves=97) 
Cross-Validation Score: 93.38 (+/- 0.06) [ 2144 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.0s finished


In [140]:
# CatBoost
from catboost import CatBoostClassifier
#!pip install catboost

model = CatBoostClassifier(
    silent=True, iterations=500,
    learning_rate=0.05, depth=9) # 99.61
#learning_rate=0.1, iterations=1000, depth=5
evaluate_model('CB', model)

params = dict(
    iterations=[10,50,100,250,500,1000],
    learning_rate=[0.01,0.05,0.1,0.5],
    depth=range(1,11,2)
) 
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Cross-Validation Score: 93.27 (+/- 0.08) [74697 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.2min finished


In [143]:
names = []
estimators = []
scores = []
stdevs = []
times = []

for key, value in models.items():
    (model, score, stdev, elapsed) = value
    names.append(key)
    estimators.append(model)
    scores.append(score)
    stdevs.append(stdev)
    times.append(elapsed)

In [144]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stdevs,
    'Time (ms)': times,
    'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

,Model,Score,Std Dev,Time (ms),Estimator
11,XGB,94.221869,0.047438,180275,"XGBClassifier(base_score=None, booster=None, c..."
7,RF,94.152472,0.091703,9996,RandomForestClassifier(random_state=42)
8,ABDT,93.938942,0.094272,1551,AdaBoostClassifier(base_estimator=DecisionTree...
6,DT,93.707261,0.098637,619,"DecisionTreeClassifier(max_depth=17, random_st..."
12,LGBM,93.384830,0.062459,2144,"LGBMClassifier(feature_fraction=0.5, max_depth..."
13,CB,93.270057,0.075435,74697,<catboost.core.CatBoostClassifier object at 0x...
2,LDA,83.329151,0.252993,951,LinearDiscriminantAnalysis()
1,LSVM,83.154056,0.360555,35470,"LinearSVC(C=0.1, random_state=42)"
0,LR,82.889279,0.243581,2271,"LogisticRegression(C=0.1, max_iter=1000, rando..."
4,GNB,82.800664,0.242912,306,GaussianNB(var_smoothing=1.0)


In [161]:
model = models['XGB'][0]
model

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.6,
              eval_metric='logloss', gamma=0.4, gpu_id=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=9,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              random_state=None, reg_alpha=1e-05, reg_lambda=None,
              scale_pos_weight=None, subsample=0.9, tree_method=None,
              use_label_encoder=False, validate_parameters=None,
              verbosity=None)

In [162]:
# treinar o melhor modelo com todos os dados
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, eval_metric='logloss',
              gamma=0.4, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=9, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, random_state=0, reg_alpha=1e-05,
              reg_lambda=1, scale_pos_weight=1, subsample=0.9,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [163]:
# gravar o modelo em disco
from joblib import dump
dump(model, '71-modelo-genero-nome.joblib')

['71-modelo-genero-nome.joblib']

## Avaliação do melhor modelo

In [164]:
# carregar um modelo previamente gravado em disco
from joblib import load
model = load('71-modelo-genero-nome.joblib')
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, eval_metric='logloss',
              gamma=0.4, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=9, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, random_state=0, reg_alpha=1e-05,
              reg_lambda=1, scale_pos_weight=1, subsample=0.9,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [165]:
# definir arquivo Pickle de entrada
#arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

df2 = pd.read_pickle(arquivo)
df2.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,15925,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,11497,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,7515,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,7370,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [166]:
df2.shape

(41146, 18)

In [167]:
X_test = df2.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y_test = df2['SEXO'] # apenas a coluna alvo

In [168]:
y_pred = model.predict(X_test)
y_pred

array([0, 1, 0, ..., 0, 1, 1])

In [169]:
from sklearn.metrics import accuracy_score

print("Accuracy score: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

Accuracy score: 94.94 %


In [170]:
from sklearn.metrics import confusion_matrix

print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

Confusion matrix:
 [[21105   785]
 [ 1299 17957]]


In [171]:
dados = pd.DataFrame({'REAL': y_test, 'PREV': y_pred}, index=X_test.index)
for col in dados.columns:
    dados[col] = dados[col].map({0: 'F', 1: 'M', 9: 'X'})
dados.head(10)

,REAL,PREV
NOME,,
MARIA,F,F
JOSE,M,M
ANA,F,F
JOAO,M,M
CARLOS,M,M
PAULO,M,M
ANTONIO,M,M
LUIZ,M,M
FRANCISCO,M,M


In [172]:
dados[dados['REAL'] != dados['PREV']].head(20)

,REAL,PREV
NOME,,
CONCEICAO,M,F
DJALMA,F,M
ESTER,M,F
UBIRAJARA,F,M
RENE,F,M
ELI,M,F
ESTHER,M,F
THAMIRES,M,F
DIONE,F,M
